# Testing con Pytest

Los objetivos de aprendizaje son:

1. Motivación.
2. Uso básico de Pytest.
3. Fixtures.
4. Marks
5. Parametrización


## Motivación

Durante el proceso de desarrollo es muy importante comprobar que el código que escribimos funciona. 

Supongamos que queremos desarrollar una función que dada una `string` la convierta a [snake_case](https://en.wikipedia.org/wiki/Snake_case)

utils.py

````python
def to_snake_case(x: str) -> str:
    return '_'.join(x.lower().split())
````

Lo siguiente será comprobar que funciona:

In [ ]:
from utils import to_snake_case

print(to_snake_case("hola mundo"))
print(to_snake_case("Curso Python"))
print(to_snake_case("notebook"))

Todo correcto, pero esta forma es muy manual y para que alguien más verifique pueda verificar los resultados es necesario que entienda cuáles deben ser las entradas y salidas.

Podríamos intentar lo siguiente:

verificar_utils.py

```python
from utils import to_snake_case

def verificar_utils():
    
    print(to_snake_case("hola mundo") == "hola_mundo")
    print(to_snake_case("Curso Python") == "curso_python")
    print(to_snake_case("notebook") == "notebook")
    
if __name__ == '__main__':
    verificar_utils()
```

Con el caracter `!` podemos ejecutar código de python desde consola:

In [ ]:
!python verificar_utils.py

Mejor, pero supongamos que alguien por accidente modifica el script.

verificar_utils_2.py

```python
from utils import to_snake_case

def verificar_utils():
    
    print(to_snake_case(2) == "hola_mundo")
    print(to_snake_case("Curso Python") == "curso_python")
    print(to_snake_case("notebook") == "notebook")
    
if __name__ == '__main__':
    verificar_utils()
```


In [ ]:
!python verificar_utils_2.py

Ha fallado y el resto verificaciones no se han ejecutado.

En este caso es fácil saber por qué a fallado, pero en bases de código más grandes en donde estamos probando muchas funciones, esta solución no es escalable.

## Uso básico de Pytest.

Es una herramienta que simplifica y automatiza el proceso de pruebas, aunque no es la única `Pytest` destaca entre por su facilidad de uso y su capacidad para gestionar escenarios complejos.


Comenzaremos por reformular nuestro script verificar.

test_utils.py

``` python
import pytest
from utils import to_snake_case

def test_dos_palabras():
    assert to_snake_case(2) == "hola_mundo"

def test_dos_palabras_mayusculas():
    assert to_snake_case("Curso Python") == "curso_python"
    
def test_una_palabra():
    assert to_snake_case("notebook") == "notebook"
```

In [ ]:
!pytest test_utils.py -vv 

De esta manera podríamos refactorizar y hacer más seguro nuestro código de la siguiente forma

version_2/utils.py
```python
def to_snake_case(x: str) -> str:
    if not isinstance(x, str):
        raise TypeError('El argumento x debe ser un string')
    return '_'.join(x.lower().split())

```

version_2/test_utils.py
``` python
import pytest
from utils import to_snake_case

def test_dos_palabras():
    assert to_snake_case("hola_mundo") == "hola_mundo"

def test_dos_palabras_mayusculas():
    assert to_snake_case("Curso Python") == "curso_python"
    
def test_una_palabra():
    assert to_snake_case("notebook") == "notebook"

def test_raises_exception_no_string_arg():
    with pytest.raises(TypeError):
        to_snake_case(5)

```

In [ ]:
!cd version_2 && pytest -vv

## Fixtures.

Son una forma de proporcionar datos (dobles de pruebas) a un test, i.e configurar el estado de las pruebas.

Los `fixtures` son funciones que pueden devolver una amplia gama de valores. Cada prueba que dependa de un `fixture` debe aceptar explícitamente ese `fixture` como argumento.

### Cuándo crear un Fixture.

Simularemos el flujo de trabajo típico de [*test-driven development*](https://en.wikipedia.org/wiki/Test-driven_development) (TDD).

Imaginemos que estamos escribiendo la función `format_data_for_display()`, Esta función procesa los datos que nos regresa un `endpoint` de una API .

Los datos que nos regresa la API representan una lista de personas, cada una con:

- Nombre
- Apellido
- Cargo. 

La función debe generar una lista de cadenas que incluyan el nombre completo de cada persona, dos puntos y su título.

version_3/utils.py
````python
from typing import List, Dict

Personas = List[Dict[str, str]]

def format_data_for_display(personas: Personas):
    # TODO: implementar el código de la función. 
    raise NotImplementedError()
````

En TDD antes de desarrollar el código escribiremos una prueba, por ejemplo:

version_3/test_format_data.py
```` python
from utils import format_data_for_display

def test_format_data_for_display():
    personas = [
        {
            "nombre": "Heber",
            "apellido": "Trujillo",
            "cargo": "Machine Learning Enineer",
        },
        {
            "nombre": "Montserrat",
            "apellido": "Navarro",
            "cargo": "Data Enineer",
        },
    ]

    assert format_data_for_display(personas) == [
        "Heber Trujillo: Machine Learning Enineer",
        "Montserrat Navarro: Data Enineer",
    ]
````

Supongamos que mientras escribimos este test, se nos ocurre que también podríamos necesitar escribir una función para transformar los datos en valores separados por comas para compartir los resultados en formato `*.csv`.

version_3/utils.py
````python
from typing import List, Dict

Personas = List[Dict[str, str]]

def format_data_for_display(personas: Personas):
    # TODO: implementar el código de la función. 
    raise NotImplementedError()

def format_data_for_csv(personas: Personas):
    # TODO: implementar el código de la función. 
    raise NotImplementedError()
````

> **Nota:** Una de las ventajas de TDD es que nos ayuda a planificar el trabajo que tenemos por hacer. 

La prueba para la función `format_data_for_csv()` se vería muy similar a la función `format_data_for_display()`, iríamos en contra del principio DRY.


version_3/test_format_data.py
```` python
from utils import format_data_for_display, format_data_for_csv

def test_format_data_for_display():
    personas = [
        {
            "nombre": "Heber",
            "apellido": "Trujillo",
            "cargo": "Machine Learning Enineer",
        },
        {
            "nombre": "Montserrat",
            "apellido": "Navarro",
            "cargo": "Data Enineer",
        },
    ]

    assert format_data_for_display(personas) == [
        "Heber Trujillo: Machine Learning Enineer",
        "Montserrat Navarro: Data Enineer",
    ]

def test_format_data_for_csv():
    personas = [
        {
            "nombre": "Heber",
            "apellido": "Trujillo",
            "cargo": "Machine Learning Enineer",
        },
        {
            "nombre": "Montserrat",
            "apellido": "Navarro",
            "cargo": "Data Enineer",
        },
    ]

    assert format_data_for_csv(personas) == "nombre,apellido,cargo\nHeber,Trujillo,Machine Learning Enineer\nMontserrat,Navarro,Data Enineer"
````

Ambas pruebas tienen que repetir la definición de la variable de `personas`.

Qué hacer si escribimos varias pruebas que hacen uso de los mismos datos?

Podemos declarar los datos repetidos en una función decorada con `@pytest.fixture` para indicar que la función es un `fixture` de pytest:

version_3/test_format_data.py
```` python
import pytest
from utils import format_data_for_display, format_data_for_csv, Personas


@pytest.fixture
def personas() -> Personas:
    return [
        {
            "nombre": "Heber",
            "apellido": "Trujillo",
            "cargo": "Machine Learning Enineer",
        },
        {
            "nombre": "Montserrat",
            "apellido": "Navarro",
            "cargo": "Data Enineer",
        },
    ]

def test_format_data_for_display(personas: Personas):
    
    assert format_data_for_display(personas) == [
        "Heber Trujillo: Machine Learning Enineer",
        "Montserrat Navarro: Data Enineer",
    ]

def test_format_data_for_csv(personas: Personas):
    assert format_data_for_csv(personas) == "nombre,apellido,cargo\nHeber,Trujillo,Machine Learning Enineer\nMontserrat,Navarro,Data Enineer"
````

### Cuando Evitar Usar Fixtures

Los `fixtures` son excelentes para extraer datos que se usan en múltiples pruebas. Sin embargo, no siempre son tan buenos para las pruebas que requieren ligeras variaciones en los datos. 

> Ensuciar las pruebas con `fixtures` no es mejor que ensuciarlas con datos u objetos simples. Incluso podría ser peor debido a la capa adicional de direccionamiento indirecto.


Como ocurre con la mayoría de las abstracciones, se necesita algo de práctica y reflexión para encontrar el nivel adecuado de uso

In [ ]:
!cd version_3 && pytest -vv

### Cómo usar fixtures a escala

A medida que creamos `fixtures`, es posible identificar algunos `fixtures` que podrían beneficiarse de una mayor modularización, i.e. definirlos en un común e importarlos.

Por ejemplo, si dos archivos `test_*.py` comparten un mismo `fixture` podríamos mover el código duplicado a un módulo general que contenga `fixtures`. Este es un buen enfoque cuando usamos un `fixture` repetidamente a lo largo de un proyecto.

`pytest` cuenta con una funcionalidad que automatiza esto, 
Si queremos que un `fixture` esté disponible en toda la suite de pruebas sin tener que importarlo en cada sitio, podemos configurar un módulo especial llamado `conftest.py`.


version_4/test_format_data.py
```` python
import pytest
from utils import format_data_for_display, format_data_for_csv, Personas

def test_format_data_for_display(personas: Personas):
    
    assert format_data_for_display(personas) == [
        "Heber Trujillo: Machine Learning Enineer",
        "Montserrat Navarro: Data Enineer",
    ]

def test_format_data_for_csv(personas: Personas):
    assert format_data_for_csv(personas) == "nombre,apellido,cargo\nHeber,Trujillo,Machine Learning Enineer\nMontserrat,Navarro,Data Enineer"
````

version_4/conftest.py
```Python 
import pytest
from utils import Personas

@pytest.fixture
def personas() -> Personas:
    return [
        {
            "nombre": "Heber",
            "apellido": "Trujillo",
            "cargo": "Machine Learning Enineer",
        },
        {
            "nombre": "Montserrat",
            "apellido": "Navarro",
            "cargo": "Data Enineer",
        },
    ]

```


In [ ]:
!cd version_4 && pytest -vv

## Marks

En una siute de pruebas grande querremos evitar ejecutar todas las pruebas cuando desarrollamos una característica pequeña. 

Además del comportamiento predeterminado de pytest para ejecutar todas las pruebas en el directorio de trabajo actual o el parámetro `--ignore` podemos filtrar de manera más granular mediante el uso de marcdores.

`pytest` nos permite definir categorías para nuestras pruebas y brinda opciones para incluir o excluir categorías cuando ejecuta la suite.

Pese a que el número de marcadores es indeterminado, los valores típicos son:

- unit: Pruebas unitarias, e.g. comprueban que una función de ejecuta correctamente.
- integration: Pruebas que integran dos o más funcionalidades, e.g. una clase con varios métodos que cambian el estado de la instancia.
- e2e: Sirven para comprobar de inicio a fin una rutina compleja.

También podemos usar otros valores como `@pytest.mark.database_access` Si algunas de nuestras pruebas requieren acceso a una base de datos.

version_5/test_format_data.py
```python 
import pytest
from utils import format_data_for_display, format_data_for_csv, Personas

@pytest.mark.unit
def test_format_data_for_display(personas: Personas):

    assert format_data_for_display(personas) == [
        "Heber Trujillo: Machine Learning Enineer",
        "Montserrat Navarro: Data Enineer",
    ]

@pytest.mark.e2e
def test_format_data_for_csv(personas: Personas):
    assert format_data_for_csv(personas) == "nombre,apellido,cargo\nHeber,Trujillo,Machine Learning Enineer\nMontserrat,Navarro,Data Enineer"
```

> **PRO Tip**: Dado que podemos llamar como queramos a nuetros `marks`, es fácil escribir mal o recordar mal el nombre de un `mark`. pytest nos advertirá sobre esto. 

Para evitarlo podríamos:

1. Añadir la configuración de inicialización de `pytest` al `project.toml`

```toml
[tool.pytest.ini_options]
markers = [
    "unit: pruebas unitarias",
    "e2e",
]
```

2. Añadir el flag `--strict-markers` a la llamada de `pytest`

In [ ]:
!cd version_5 && pytest -vv -m unit

## Parametrización


Antes vimos cómo usar `fixtures` en `pytest` para reducir la duplicación de código, y como los `fixtures` no son tan útiles cuando tenemos varias pruebas con inputs ligeramente diferentes.

Cuando los inputs son ligeramente distintos podemos parametrizar una sola definición de prueba y `pytest` creará variantes de la prueba con los parámetros que especifiquemos.

Imaginemos que tenemos los siguientes archivos:

version_6/utils.py
```python
import re

def es_palindrome(s: str)->bool:
    s = s.lower().replace(" ", "")
    s = re.sub(r'[^\w\s]', '', s)
    return s == s[::-1]
```

version_6/test_palindrome.py
```python
from utils import es_palindrome

def test_es_palindrome_string_vacia():
    assert es_palindrome("")

def test_es_palindrome_caracter():
    assert es_palindrome("a")

def test_es_palindrome_mayusculas_minusculas():
    assert es_palindrome("Bob")

def test_es_palindrome_con_espacios():
    assert es_palindrome("anita lava la tina")

def test_es_palindrome_con_signos():
    assert es_palindrome("anita lava la tina?")

def test_no_es_palindrome():
    assert not es_palindrome("abc")

def test_no_es_palindrome_por_poco():
    assert not es_palindrome("abab")
```

In [ ]:
!cd version_6 && pytest -vv

Todas las funciones tiene el patrón:

```python
def test_es_palindrome_<caso particular>():
    assert es_palindrome("<caso particular>")
```



Muy repetitivo. Puedemos usar @pytest.mark.parametrize() reducir significativamente el código de prueba:


version_7/test_palindrome.py
```` python
from utils import es_palindrome

@pytest.mark.parametrize("palindrome", [
    "",
    "a",
    "Bob",
    "anita lava la tina",
    "anita lava la tina?",
])
def test_es_palindrome(palindrome):
    assert es_palindrome(palindrome)

@pytest.mark.parametrize("no_palindrome", [
    "abc",
    "abab",
])
def test_no_es_palindrome(no_palindrome):
    assert not es_palindrome(no_palindrome)
```` 

El primer argumento para `parametrize()` es el nombre del parámetro que usaremos para todos los posibles valores. El segundo argumento es una lista de tuplas o valores individuales que representan los valores del parámetro. Podríamos simplificar aún más la expresión

version_8/test_palindrome.py
```` python
from utils import es_palindrome

@pytest.mark.parametrize("palindrome, resultado_esperado", [
    ("", True),
    ("a", True),
    ("Bob", True),
    ("Never odd or even", True),
    ("Do geese see God?", True),
    ("abc", False),
    ("abab", False),
])
def test_es_palindrome(palindrome, resultado_esperado):
    assert es_palindrome(palindrome) == resultado_esperado
```` 

In [ ]:
!cd version_7 && pytest -vv